In [1]:
!python -m nltk.downloader stopwords

/usr/lib/python3.8/runpy.py:127: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/bandala/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [15]:
import networkx as nx
from nltk.corpus import stopwords
from tqdm import tqdm
from networkx.algorithms import community
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt
import plotly.express as px
from random import random    

# Semantic network
A semantic network, or frame network is a knowledge base that represents semantic relations between concepts in a network. This is often used as a form of knowledge representation. It is a directed or undirected graph consisting of vertices, which represent concepts, and edges, which represent semantic relations between concepts, mapping or connecting semantic fields. A semantic network may be instantiated as, for example, a graph database or a concept map. Typical standardized semantic networks are expressed as semantic triples.

Semantic networks are used in natural language processing applications such as semantic parsing
 and word-sense disambiguation. Semantic networks can also be used as a method to analyze large texts and identify the main themes and topics (e.g., of social media posts), to reveal biases (e.g., in news coverage), or even to map an entire research field.

In [16]:
# initialize stemmer
ps = PorterStemmer()

In [4]:
def load_file(filename):
    with open(filename,'r') as f:
        text = f.readlines()
    return text

In [18]:
def preprocess(line):
    stop_words = set(stopwords.words('english'))
    line = [item.lower() for item in line if not item.lower() in stop_words]
    #print(line)
    stemmed_line = [ps.stem(item) for item in line]
    #print(stemmed_line)
    return stemmed_line

In [6]:
def create_graph(text):
    word_list = []
    G = nx.Graph()
    pbar = tqdm(total=len(text))
    for line in text:
        line = (line.strip()).split()
        line = preprocess(line)
        for i, word in enumerate(line):
            if i != len(line)-1:
                word_a = word
                word_b = line[i+1]
                if word_a not in word_list:
                    word_list.append(word_a)
                if word_b not in word_list:
                    word_list.append(word_b)
                if G.has_edge(word_a,word_b):
                    G[word_a][word_b]['weight'] += 1
                else:
                    G.add_edge(word_a,word_b, weight = 1)
        pbar.update(1)
    pbar.close()
    return G

In [7]:
def calculate_central_nodes(text_network , max_nodes = -1):
    bc = (nx.betweenness_centrality(text_network,weight='weight'))
    #print(bc)
    nx.set_node_attributes(text_network, bc, 'betweenness')
    bc_threshold = sorted(bc.values(), reverse=True)[max_nodes]
    to_keep = [n for n in bc if bc[n] > bc_threshold]
    filtered_network = text_network.subgraph(to_keep)
    return filtered_network

In [8]:
def plot_betweenness_centrality(text_network,max_nodes):
    bc = (nx.betweenness_centrality(text_network,weight='weight'))
    sorted_bc = dict(sorted(bc.items(),reverse=True, key=lambda item: item[1]))
    nodes = list(sorted_bc.keys())[:max_nodes]
    values = list(sorted_bc.values())[:max_nodes]
    fig = px.bar( x=nodes, y=values)
    fig.show()

In [9]:
def plot_degree_centrality(text_network,max_nodes , normalized =True):
    if normalized : 
        bc = (nx.degree_centrality(text_network))
    else : 
        bc = dict(text_network.degree())
    #print(bc)
    sorted_bc = dict(sorted(bc.items(),reverse=True, key=lambda item: item[1]))
    nodes = list(sorted_bc.keys())[:max_nodes]
    values = list(sorted_bc.values())[:max_nodes]
    fig = px.bar( x=nodes, y=values)
    fig.show()

In [10]:
def scatter_centralities(text_network) :
    deg = (nx.degree_centrality(text_network))
    bc = (nx.betweenness_centrality(text_network,weight='weight'))
    values_bc =  list(bc.values())
    values_deg =  list(deg.values())
    labels = list(bc.keys())
   # print(labels)
    fig = px.scatter( x=values_deg, y=values_bc,labels = labels)
    fig.show()

In [11]:
def create_and_assign_communities(text_network):
    communities_generator = community.girvan_newman(text_network)
    top_level_communities = next(communities_generator)
    next_level_communities = next(communities_generator)
    communities = {}
    for community_list in next_level_communities:
        for item in community_list:
            communities[item] = next_level_communities.index(community_list)
    return communities

In [12]:
def draw_final_graph(text_network,communities ,with_size=True ):
    pos = nx.spring_layout(text_network,scale=2)
    color_list = []
    color_map = []
    community_count = max(communities.values())
    for i in range(0,community_count+1):
        color_list.append((random(), random(), random()))
    for node in text_network:
        color_index = communities[node]
        color_map.append(color_list[color_index])
    betweenness = nx.get_node_attributes(text_network,'betweenness')
    betweenness = [x*1000 for x in betweenness.values()]
    if with_size : 
         nx.draw(text_network,with_labels=True,node_size=betweenness,font_size=8,node_color=color_map,edge_cmap=plt.cm.Blues)
    else :  nx.draw(text_network,with_labels=True,font_size=10,node_color=color_map,edge_cmap=plt.cm.Blues)
    plt.draw()
    plt.show()

In [23]:
text = load_file('semantic_query.txt')
text_network = create_graph(text)
plot_betweenness_centrality(text_network,max_nodes =20,)
#scatter_centralities(text_network)

100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 1957.21it/s]


ValueError: Cannot accept list of column references or list of columns for both `x` and `y`.

In [152]:
text = load_file('./test.txt')
text_network = create_graph(text)
plot_betweenness_centrality(text_network,max_nodes =20 ,)
#scatter_centralities(text_network)


100%|██████████| 3/3 [00:00<00:00, 527.12it/s]


In [146]:
text = load_file('./test.txt')
text_network = create_graph(text)
plot_degree_centrality(text_network,max_nodes =20)


100%|██████████| 1/1 [00:00<00:00, 540.22it/s]


In [145]:
text = load_file('./test.txt')
text_network = create_graph(text)

scatter_centralities(text_network)


100%|██████████| 1/1 [00:00<00:00, 551.01it/s]


In [21]:
text = load_file('semantic_query.txt')
text_network = create_graph(text)
text_network = calculate_central_nodes(text_network,max_nodes = 10)
communities = create_and_assign_communities(text_network)

nx.write_gml(text_network, "text_network.gml")
draw_final_graph(text_network,communities,with_size=True)

100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 986.90it/s]


IndexError: list index out of range

In [140]:
communities

{'sort': 0,
 'approach': 0,
 'query': 0,
 'might': 0,
 'learning.': 0,
 'modern': 0,
 'detail': 0,
 'broad': 0,
 'idea': 0,
 '"query': 0,
 'outline': 0,
 'touch': 0,
 'frameworks."': 0,
 'scenario': 0,
 'machine': 0,
 'problem.': 0,
 'categories,': 0,
 'theoretical': 0,
 'strength': 0,
 'algorithm': 0,
 'weakness': 0,
 'also': 0,
 'well-motivated': 0,
 'learning,': 0,
 'four': 0,
 'formulated,': 0,
 'active': 0,
 'learning': 0,
 'behind': 0,
 'organized': 0,
 'overview': 0,
 'nature': 0,
 'foundations': 0,
 'several': 0,
 'selection': 0,
 'conclude': 0,
 'many': 0,
 'general': 1,
 'unlabeled': 1,
 'easy': 1,
 'instance': 1,
 'come': 1,
 'mining': 1,
 'difficult,': 1,
 'pose': 1,
 'better': 1,
 'already': 1,
 'by,': 1,
 'may': 1,
 'expensive': 1,
 'learns.': 1,
 'human': 1,
 'data': 1,
 '(e.g.,': 1,
 'book': 1,
 'perform': 1,
 'understand': 1,
 '"queries,"': 1,
 'form': 1,
 'annotator)': 1,
 'allow': 1,
 'learner': 1,
 'usually': 1,
 'obtain.': 1,
 'introduction': 1,
 'less': 1,
 'label